<a href="https://colab.research.google.com/github/tbazzi/Data-Science---Fullstack-Bootcamp/blob/master/S4_5_Titanic_Stacking_SOLUTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faire du Stacking avec Titanic 

On va faire notre premier stacking ensemble. Voyons comment cela peut améliorer notre score sur le dataset du Titanic 

1. Importez les librairies usuelles 

In [0]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

2. Importez le dataset dans un DataFrame 

In [0]:
dataset = pd.read_csv("https://raw.githubusercontent.com/JedhaBootcamp/FULL_STACK_12_WEEK_PROGRAM/master/S0.4%20--%20R%C3%A9duction%20de%20Dimensions%20%26%20Boosting/S4-4%20--%20Ensemble%20Learning/Dataset/titanic.csv?token=AFB53D7NAVKN2AUUUC3WL525UBRYC")

dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


3. Remplacez la colonne 

```name```
```Ticket```

Par le nombre de caractères qu'il y a dans la cellule 

In [0]:
dataset["Ticket"] = dataset["Ticket"].apply(len)
dataset["Name"] = dataset["Name"].apply(len)

dataset.head()

TypeError: ignored

4. Créez une nouvelle colonne ```FamilySize``` qui sera la somme des colonnes  ```SibSp``` & ```Parch```

In [0]:
dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"]
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,23,male,22.0,1,0,9,7.2500,NaN,S,1
1,2,1,1,51,female,38.0,1,0,8,71.2833,C85,C,1
2,3,1,3,22,female,26.0,0,0,16,7.9250,NaN,S,0
3,4,1,1,44,female,35.0,1,0,6,53.1000,C123,S,1
4,5,0,3,24,male,35.0,0,0,6,8.0500,NaN,S,0


5. Créez une nouvelle colonne ```IsAlone``` qui indique si la personne a une famille ou non 

In [0]:
dataset["IsAlone"] = dataset["FamilySize"].apply(lambda x:0 if x>0 else 1)
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0,3,23,male,22.0,1,0,9,7.2500,NaN,S,1,0
1,2,1,1,51,female,38.0,1,0,8,71.2833,C85,C,1,0
2,3,1,3,22,female,26.0,0,0,16,7.9250,NaN,S,0,1
3,4,1,1,44,female,35.0,1,0,6,53.1000,C123,S,1,0
4,5,0,3,24,male,35.0,0,0,6,8.0500,NaN,S,0,1


6. Dans la colonne ```Cabin``` Faites en sorte de ne garder uniquement la première lettre de la cabine

In [0]:
dataset["Cabin"] = dataset["Cabin"].apply(lambda x:x[0] if type(x) == str else "NoCabin")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0,3,23,male,22.0,1,0,9,7.2500,NoCabin,S,1,0
1,2,1,1,51,female,38.0,1,0,8,71.2833,C,C,1,0
2,3,1,3,22,female,26.0,0,0,16,7.9250,NoCabin,S,0,1
3,4,1,1,44,female,35.0,1,0,6,53.1000,C,S,1,0
4,5,0,3,24,male,35.0,0,0,6,8.0500,NoCabin,S,0,1


7. Faites une interpolation linéaire pour remplacer les valeurs manquantes dans la colonne ```Age```

In [0]:
dataset["Age"] = dataset["Age"].interpolate()
dataset.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked        True
FamilySize     False
IsAlone        False
dtype: bool

8. Comptez le nombre de NaN qu'il y a dans la colonne Embarked et adoptez la stratégie qui vous semble le plus adapté pour gérer ces valeurs 

In [0]:
dataset.isnull().sum()/len(dataset)

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.000000
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.000000
Embarked       0.002245
FamilySize     0.000000
IsAlone        0.000000
dtype: float64

In [0]:
dataset = dataset.dropna()

In [0]:
dataset.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
FamilySize     False
IsAlone        False
dtype: bool

9. Dummyfiez les variables catégoriques 

In [0]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,FamilySize,IsAlone,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_NoCabin,Cabin_T,Embarked_Q,Embarked_S
0,1,0,3,23,22.0,1,0,9,7.2500,1,0,1,0,0,0,0,0,0,1,0,0,1
1,2,1,1,51,38.0,1,0,8,71.2833,1,0,0,0,1,0,0,0,0,0,0,0,0
2,3,1,3,22,26.0,0,0,16,7.9250,0,1,0,0,0,0,0,0,0,1,0,0,1
3,4,1,1,44,35.0,1,0,6,53.1000,1,0,0,0,1,0,0,0,0,0,0,0,1
4,5,0,3,24,35.0,0,0,6,8.0500,0,1,1,0,0,0,0,0,0,1,0,0,1


10. Séparez votre dataset en X & y qui sont respectivement les variables explicatives et la variable cible 

In [0]:
X = dataset.iloc[:,2:]
y = dataset.iloc[:, dataset.columns=="Survived"]

11. Faites un train_test_split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

12. Normalisez les données

In [0]:


from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

13. Faites une première prédiction grâce à une régression logistique et regardez votre score 

In [0]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
print(classifier.score(X_test, y_test))

0.7957957957957958
0.8385650224215246


14. Faites une prédiction pour tout votre dataset X et concaténez ces prédiction dans un dataframe qu'on appellera X_new 

In [0]:
prediction_train = classifier.predict_proba(X_train)[:,0]
X_train_new = pd.concat([pd.DataFrame(X_train), pd.DataFrame(prediction_train)], axis=1)
prediction_test =  classifier.predict_proba(X_test)[:,0]
X_test_new = pd.concat([pd.DataFrame(X_test), pd.DataFrame(prediction_test)], axis=1)
X_train_new.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,0
0,0.819431,1.355805,1.311732,0.453131,3.600111,1.224790,0.073592,2.036865,-1.237869,-1.354268,-0.224662,-0.259437,-0.197488,-0.184828,-0.117041,-0.067267,0.522898,-0.038778,-0.326056,0.650444,0.702596
1,-0.385295,-1.075683,-0.110029,-0.475439,0.880118,-0.252946,0.044583,0.087796,-1.237869,0.738406,-0.224662,-0.259437,-0.197488,-0.184828,-0.117041,-0.067267,0.522898,-0.038778,-0.326056,0.650444,0.799316
2,-1.590021,0.471628,-0.465469,-0.475439,0.880118,0.485922,0.685054,0.087796,-1.237869,0.738406,-0.224662,-0.259437,5.063596,-0.184828,-0.117041,-0.067267,-1.912418,-0.038778,-0.326056,-1.537412,0.123454
3,0.819431,-1.075683,-1.425157,-0.475439,-0.479879,-0.991814,-0.499104,-0.561894,0.807840,0.738406,-0.224662,-0.259437,-0.197488,-0.184828,-0.117041,-0.067267,0.522898,-0.038778,-0.326056,-1.537412,0.784418
4,0.819431,-0.191505,1.027380,-0.475439,-0.479879,-0.252946,-0.481788,-0.561894,0.807840,0.738406,-0.224662,-0.259437,-0.197488,-0.184828,-0.117041,-0.067267,0.522898,-0.038778,-0.326056,0.650444,0.948687


15. Importez Adaboost et entrainez votre nouveau modèle sur ```X_new```

In [0]:
#from sklearn.ensemble import AdaBoostClassifier
lg = LogisticRegression()
lg.fit(X_train_new, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

16. Regardez votre nouveau score 

In [0]:
lg.score(X_test_new, y_test)

0.8385650224215246